In [3]:
%%capture
!pip install JAEN -qU

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchinfo import summary
from JAEN.utils import plot_training_results

# device 설정 (GPU가 사용 가능하면 GPU로, 그렇지 않으면 CPU 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [46]:
import os

# JAEN 모듈 불러오기
from JAEN.datasets import load_titanic

# 데이터 파일이 저장된 경로 찾기
jaen_path = os.path.dirname(load_titanic.__code__.co_filename)
train_path = os.path.join(jaen_path, "data", "00", "train_loader.pt")
test_path = os.path.join(jaen_path, "data", "00", "test_loader.pt")

# torch.load()를 직접 실행 (weights_only=False 설정)
train_loader = torch.load(train_path, weights_only=False)
test_loader = torch.load(test_path, weights_only=False)

# 데이터 확인
print(train_loader)
print(test_loader)


In [18]:
model = nn.Sequential(
    nn.Linear(7, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)

# 모델 인스턴스 생성
model = model.to(device)
summary(model, (32, 7))

# 32 → 한 번에 모델에 입력하는 샘플 개수 (Batch Size)
# 7 → 각 샘플의 특성(feature) 개수

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 1]                   --
├─Linear: 1-1                            [32, 32]                  256
├─ReLU: 1-2                              [32, 32]                  --
├─Linear: 1-3                            [32, 1]                   33
├─Sigmoid: 1-4                           [32, 1]                   --
Total params: 289
Trainable params: 289
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

## nn.Module 기반 신경망 모델 구성

In [16]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 32)
        self.fc2 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# 모델 인스턴스 생성
model = DNN().to(device)
summary(model, (32, 2))

Layer (type:depth-idx)                   Output Shape              Param #
DNN                                      [32, 1]                   --
├─Linear: 1-1                            [32, 32]                  96
├─ReLU: 1-2                              [32, 32]                  --
├─Linear: 1-3                            [32, 1]                   33
├─Sigmoid: 1-4                           [32, 1]                   --
Total params: 129
Trainable params: 129
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [17]:
X = torch.tensor([1.0, 2.0])
model.forward(X)

tensor([0.4591], grad_fn=<SigmoidBackward0>)

In [19]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7, 32)
        self.fc2 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# 모델 인스턴스 생성
model = DNN().to(device)
summary(model, (32, 7))

Layer (type:depth-idx)                   Output Shape              Param #
DNN                                      [32, 1]                   --
├─Linear: 1-1                            [32, 32]                  256
├─ReLU: 1-2                              [32, 32]                  --
├─Linear: 1-3                            [32, 1]                   33
├─Sigmoid: 1-4                           [32, 1]                   --
Total params: 289
Trainable params: 289
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

## 모델 학습 코드

In [35]:
X, y = list(train_loader)[0]

In [36]:
X.shape, y.shape

(torch.Size([32, 7]), torch.Size([32, 1]))

In [38]:
criterion = nn.BCELoss() # 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.01) # 옵티마이저

In [39]:
outputs = model(X) # 순전파
outputs.shape

torch.Size([32, 1])

In [41]:
loss = criterion(outputs, y) # 손실 계산
loss

tensor(0.7176, grad_fn=<BinaryCrossEntropyBackward0>)

In [42]:
loss.backward() # 역전파

In [43]:
optimizer.step() # 가중치(파라미터) 업데이트

#### 모델 학습 함수 구현

In [45]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()  # 모델을 학습모드로 설정
    
    running_loss = 0.0  # 미니 배치 별 loss 값을 누적할 변수

    for data, labels in train_loader: # 미니 배치 별 파라미터 업데이트 수행
        data, labels = data.to(device), labels.to(device)  # 미니 배치 별 데이터와 레이블 장치 할당

        # 순전파
        outputs = model(data)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 기울기 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 파라미터 업데이트
        optimizer.step()

        # 손실 누적
        running_loss += loss.item()

    # 현재 Epoch의 평균 손실 값 계산 및 반환
    return running_loss / len(train_loader)

#### 모델 평가 함수 구현

In [47]:
def evaluate(model, test_loader, criterion, device):
    model.eval()  # 모델을 평가 모드로 설정

    running_loss = 0.0  # 미니 배치 별 loss값을 누적할 변수

    with torch.no_grad():  # 평가 중에는 기울기 계산을 하지 않음
        for data, labels in test_loader:  # 미니 배치 별 손실 계산
            data, labels = data.to(device), labels.to(device)  # 미니 배치 별 데이터와 레이블 장치 할당

            # 순전파
            outputs = model(data)

            # 손실 계산
            loss = criterion(outputs, labels)

            # 손실 누적
            running_loss += loss.item()

        # 현재 Epoch의 평균 손실 값 계산 및 반환
        return running_loss / len(test_loader)

#### 학습 및 평가

In [48]:
train_losses = []
test_losses = []

# 학습 횟수 만큼 반복
for epoch in range(100):

    # 모델 학습(학습데이터)
    train_loss = train(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    # 모델 평가 (평가데이터)
    test_loss = evaluate(model, test_loader, criterion, device)
    test_losses.append(test_loss)

    print(f'Epoch {epoch+1} Train Loss : {train_loss} Test Loss : {test_loss}')

Epoch 1 Train Loss : 0.5642812640770621 Test Loss : 0.4513463228940964
Epoch 2 Train Loss : 0.45619592718456103 Test Loss : 0.43154627084732056
Epoch 3 Train Loss : 0.4314484207526497 Test Loss : 0.4223646620909373
Epoch 4 Train Loss : 0.4192415773868561 Test Loss : 0.42759112517038983
Epoch 5 Train Loss : 0.4181118905544281 Test Loss : 0.43139924108982086
Epoch 6 Train Loss : 0.40989506115084107 Test Loss : 0.4316738893588384
Epoch 7 Train Loss : 0.4107933549777321 Test Loss : 0.4388998746871948
Epoch 8 Train Loss : 0.40910309293995734 Test Loss : 0.43893905480702716
Epoch 9 Train Loss : 0.40839523141798767 Test Loss : 0.44245144228140515
Epoch 10 Train Loss : 0.4018754278836043 Test Loss : 0.4526133785645167
Epoch 11 Train Loss : 0.39829807048258575 Test Loss : 0.44263140857219696
Epoch 12 Train Loss : 0.40014003217220306 Test Loss : 0.4453007827202479
Epoch 13 Train Loss : 0.41764474368613697 Test Loss : 0.44291900595029193
Epoch 14 Train Loss : 0.3944794555073199 Test Loss : 0.4457